## Outline
* Sources:
    * https://www.kaggle.com/kingburrito666/cannabis-strains/downloads/cannabis-strains.zip/9
    * https://www.kaggle.com/tictactouka/cannabis
    
*Steps*
* Extract data into SQLite file, read tables into DataFrames with extracted columns
    * Intermediary step: transform those DFs by selecting columns, renaming, and possibly joining based on the foreign key relationships
* Extract data CSV file, read table into DataFrame with extracted columns
* Create 3 new DFs with columns from the extracted DFs
* Load the resulting DataFrame into postgres DB

## Final Table columns:
* Id (serial)
* Strain (sql inner join with csv)
* Type (csv Type)
* Breeder
* Rating_1 (csv - community rating)
* Rating_2 (sql - medical effect rating)
* Effects (csv - 
* Medical Effects
* Flavor
* Description

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Process SQLite File
* For reference: ~/classwork/10-Advanced-Data-Storage-and-Retrieval/2/Activities/03-Ins_Basic_Updating/Solved/Ins_Basic_Updating.ipynb

### Import SQLite file

In [2]:
# Create the connection engine
engine = create_engine("sqlite:///Resources/strains.sqlite")
conn = engine.connect()

### Extract Strains table into DataFrame
* Extract columns:
    * id (fk for MedicalEffects), name, breeder (fk for Breeder id), 
* Rename into this:
    * strain_id, strain_name, breeder_id

In [3]:
strains = pd.read_sql("SELECT id AS strain_id, name as strain_name, breeder as breeder_id FROM Strains",conn)

In [4]:
strains.head()

,strain_id,strain_name,breeder_id
0,1,Af-Pak,1
1,2,00 Cheese,2
2,3,Alien BubbleGum,3
3,4,Cherry OG Kush,4
4,5,Ak 420,5


### Extract Breeder Table into DataFrame
* Extract these columns:
    * id, name
* Rename into this format:
    * breeder_id, breeder_name

In [8]:
breeders = pd.read_sql("SELECT id as breeder_id, name as breeder_name FROM Breeders", conn)

In [9]:
breeders.head()

,breeder_id,breeder_name
0,1,210Beans
1,2,00 Seeds Bank
2,3,207 Seeds
3,4,420 Seeds
4,5,420 Genetics


### Transform: join breeder table and strains together

In [17]:
# join strains with breeders on breeder id
strains_breeders = pd.merge(strains,breeders,how='inner',on='breeder_id')

In [23]:
strains_breeders.head()

,strain_id,strain_name,breeder_id,breeder_name
0,1,Af-Pak,1,210Beans
1,8,BlackberryKush,1,210Beans
2,13,Pak-Man,1,210Beans
3,2,00 Cheese,2,00 Seeds Bank
4,12,00 Kush,2,00 Seeds Bank


In [26]:
#Number of strains produced by more than 1 breeder
num_dups = len(strains_breeders.strain_name) - strains_breeders.strain_name.nunique()
num_dups

1672

### Extract MedicalEffects Table into DataFrame
* Extract these columns:
    * strainid, name, info, rating
* And rename them into this format:
    * strainid, medical_effect, medical_info, medical_rating

In [6]:
medical_effects = pd.read_sql("SELECT strain_id, name as medical_effect, info as medical_info, rating as medical_rating FROM MedicalEffects", conn)

In [7]:
medical_effects.head()

,strain_id,medical_effect,medical_info,medical_rating
0,6,Anorexia and Cachexia,Affects / helps even in small doses very well ...,4.0
1,6,Autoimmune Diseases and Inflammation,Affects / helps even in small doses very well ...,4.0
2,7,Psychiatric Symptoms,Affects / helps even in small doses extremly w...,5.0
3,12,Autoimmune Diseases and Inflammation,Affects / helps even in small doses extremly w...,5.0
4,19,Pain,Affects / helps even in small doses very well ...,4.0


### Transform: join MedicalEffects and strains together

In [30]:
strains_medical = pd.merge(strains,medical_effects,how='inner',on='strain_id')

In [32]:
strains_medical.drop(['breeder_id'],axis=1)

,strain_id,strain_name,medical_effect,medical_info,medical_rating
0,6,AK-48,Anorexia and Cachexia,Affects / helps even in small doses very well ...,4.0
1,6,AK-48,Autoimmune Diseases and Inflammation,Affects / helps even in small doses very well ...,4.0
2,7,Austrian Queen,Psychiatric Symptoms,Affects / helps even in small doses extremly w...,5.0
3,12,00 Kush,Autoimmune Diseases and Inflammation,Affects / helps even in small doses extremly w...,5.0
4,19,Ambulance,Pain,Affects / helps even in small doses very well ...,4.0
5,35,Actual C99 Cinderella,Autoimmune Diseases and Inflammation,Affects / helps even in small doses extremly w...,5.0
6,81,Bazinga,Dependency and Withdrawal,Affects / helps only in heavy doses extremly w...,5.0
7,100,Bangi Haze,Anorexia and Cachexia,Affects / helps even in small doses extremly w...,5.0
8,100,Bangi Haze,Spasticity,Affects / helps in average doses very well at ...,4.0
9,100,Bangi Haze,Movement Disorders,Affects / helps only in heavy doses extremly w...,5.0


# Extract data from cannabis.csv 

### Convert csv to DataFrame

In [39]:
file = 'Resources/cannabis.csv'
cannabis_df = pd.read_csv(file)

### Create processed DataFrame
* Extract columns: Strain, Rating, Effects, Flavor, Description
* Rename columns: strain_name, community_rating, effects, flavor, description

In [40]:
cannabis_cols = ["Strain", "Type", "Rating", "Effects", "Flavor", "Description"]
cannabis_transformed = cannabis_df[cannabis_cols].copy()

In [41]:
cannabis_transformed = cannabis_transformed.rename(columns={"Strain": "strain_name",
                                                            "Type":"type",
                                                            "Rating":"community_rating",
                                                            "Effects": "community_effects",
                                                            "Flavor":"flavor",
                                                            "Description":"description"
                                                           })

In [43]:
cannabis_transformed.drop_duplicates("strain_name", inplace=True)

In [44]:
cannabis_transformed = cannabis_transformed.reset_index()

In [45]:
cannabis_transformed=cannabis_transformed.rename(columns={'index':'id'})

In [59]:
cannabis_transformed['strain_name']=cannabis_transformed['strain_name'].str.replace('-',' ')

In [61]:
csv_strains = cannabis_transformed[['strain_name']]
csv_effects = cannabis_transformed[['strain_name','community_effects']]
csv_ratings = cannabis_transformed[['strain_name','community_rating']]

# Transform resulting DataFrames into 3 new ones:
* strain_effects: strain_name, community_effects, medical_effects
* strain_ratings: strain_name, community_rating, medical_rating
* strain_description: strain_name, type, flavor, description

In [62]:
# first find the strain overlaps between 2 data sources
sql_strains = strains.drop(['strain_id','breeder_id'],axis=1)

In [63]:
strains_combined = pd.merge(sql_strains,csv_strains, on='strain_name',how='inner').drop_duplicates()

In [64]:
strains_combined

,strain_name
0,Grape Ape
3,Amnesia Haze
22,Blackberry Haze
23,Amnesia
43,Apple Jack
45,Afghan Skunk
48,Big White
51,Blueberry
72,Hawaiian Haze
73,Master Kush


# Create database and populate

### Create database connection

In [ ]:
connection_string = "postgres:postgres@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

### Load DataFrames into DB

In [ ]:
#starter code, must edit
premise_transformed.to_sql(name='premise', con=engine, if_exists='append', index=True)